In [1]:
# ---------------------------------------------------------
# Resolution Theorem Prover with Step-by-Step Trace
# ---------------------------------------------------------

import itertools

def is_variable(token):
    return token[0].islower()

def parse_literal(lit):
    neg = lit.startswith("~")
    if neg:
        lit = lit[1:]
    pred = lit.split("(")[0]
    args = lit.split("(")[1][:-1].split(",")
    args = tuple(a.strip() for a in args)
    return (pred, args, neg)

def negate(lit):
    pred, args, neg = lit
    return (pred, args, not neg)

def substitute(term, theta):
    if is_variable(term) and term in theta:
        return substitute(theta[term], theta)
    return term

def apply_substitution(literal, theta):
    pred, args, neg = literal
    new_args = tuple(substitute(a, theta) for a in args)
    return (pred, new_args, neg)

def unify_terms(a, b, theta):
    a = substitute(a, theta)
    b = substitute(b, theta)

    if a == b:
        return theta
    if is_variable(a):
        theta[a] = b
        return theta
    if is_variable(b):
        theta[b] = a
        return theta
    return None

def unify_literals(l1, l2):
    pred1, args1, neg1 = l1
    pred2, args2, neg2 = l2

    if pred1 != pred2:
        return None
    if neg1 == neg2:
        return None
    if len(args1) != len(args2):
        return None

    theta = {}

    for a, b in zip(args1, args2):
        theta = unify_terms(a, b, theta)
        if theta is None:
            return None

    return theta

def resolve(c1, c2):
    resolvents = []
    for lit1 in c1:
        for lit2 in c2:
            theta = unify_literals(lit1, lit2)
            if theta is not None:
                new_clause = set()

                for l in c1:
                    if l != lit1:
                        new_clause.add(apply_substitution(l, theta))
                for l in c2:
                    if l != lit2:
                        new_clause.add(apply_substitution(l, theta))

                resolvents.append(frozenset(new_clause))

    return resolvents

def pretty_clause(cl):
    if len(cl) == 0:
        return "{}"
    def lit_to_str(l):
        pred, args, neg = l
        return ("~" if neg else "") + pred + "(" + ",".join(args) + ")"
    return "{ " + ", ".join(lit_to_str(l) for l in cl) + " }"

# ---------------------------------------------------------
# Knowledge base clauses (CNF)
# ---------------------------------------------------------

clauses = [
    # 1. John likes all food -> ¬Food(x) ∨ Likes(John,x)
    frozenset([("Food", ("x",), True), ("Likes", ("John","x"), False)]),

    # 2. Apple, Vegetables are food
    frozenset([("Food", ("Apple",), False)]),
    frozenset([("Food", ("Vegetables",), False)]),

    # 3. Eats(x,y) ∧ ¬Killed(x) -> Food(y)
    frozenset([("Killed", ("x",), False), ("Eats",("x","y"), True), ("Food",("y",), False)]),

    # 4. Anil eats peanuts, Anil alive
    frozenset([("Eats", ("Anil","Peanuts"), False)]),
    frozenset([("Alive", ("Anil",), False)]),

    # 5. ∀y (Eats(Anil,y) -> Eats(Harry,y))
    frozenset([("Eats",("Anil","y"), True), ("Eats",("Harry","y"), False)]),

    # 6. Alive(x) → ¬Killed(x)
    frozenset([("Alive",("x",), True), ("Killed",("x",), True)]),

    # 7. ¬Killed(x) → Alive(x)
    frozenset([("Killed",("x",), False), ("Alive",("x",), False)]),

    # Negated goal: ¬Likes(John, Peanuts)
    frozenset([("Likes", ("John","Peanuts"), True)])
]

# ---------------------------------------------------------
# Resolution loop
# ---------------------------------------------------------

def resolution(clauses):
    new = set()
    iteration = 1

    while True:
        pairs = list(itertools.combinations(clauses, 2))

        for (ci, cj) in pairs:
            resolvents = resolve(ci, cj)

            for r in resolvents:
                print(f"Iteration {iteration}: Resolving")
                print("  ", pretty_clause(ci))
                print("  ", pretty_clause(cj))
                print("  =>", pretty_clause(r), "\n")

                if len(r) == 0:
                    print("❌ Empty clause derived — PROOF COMPLETE!")
                    return True

                new.add(r)
                iteration += 1

        if new.issubset(set(clauses)):
            print("No new clauses. Resolution ends with failure.")
            return False

        for c in new:
            clauses.append(c)


resolution(clauses)


Iteration 1: Resolving
   { Likes(John,x), ~Food(x) }
   { Food(Apple) }
  => { Likes(John,Apple) } 

Iteration 2: Resolving
   { Likes(John,x), ~Food(x) }
   { Food(Vegetables) }
  => { Likes(John,Vegetables) } 

Iteration 3: Resolving
   { Likes(John,x), ~Food(x) }
   { ~Eats(x,y), Killed(x), Food(y) }
  => { ~Eats(y,y), Killed(y), Likes(John,y) } 

Iteration 4: Resolving
   { Likes(John,x), ~Food(x) }
   { ~Likes(John,Peanuts) }
  => { ~Food(Peanuts) } 

Iteration 5: Resolving
   { ~Eats(x,y), Killed(x), Food(y) }
   { Eats(Anil,Peanuts) }
  => { Food(Peanuts), Killed(Anil) } 

Iteration 6: Resolving
   { ~Eats(x,y), Killed(x), Food(y) }
   { Eats(Harry,y), ~Eats(Anil,y) }
  => { Food(y), ~Eats(Anil,y), Killed(Harry) } 

Iteration 7: Resolving
   { ~Eats(x,y), Killed(x), Food(y) }
   { ~Alive(x), ~Killed(x) }
  => { ~Eats(x,y), ~Alive(x), Food(y) } 

Iteration 8: Resolving
   { Eats(Anil,Peanuts) }
   { Eats(Harry,y), ~Eats(Anil,y) }
  => { Eats(Harry,Peanuts) } 

Iteration 9: Resol

True

In [3]:
from copy import deepcopy

def negate_literal(lit):
    return lit[1:] if lit.startswith("¬") else "¬" + lit

def resolve(ci, cj):
    resolvents = []
    for li in ci:
        for lj in cj:
            if li == negate_literal(lj):
                new_clause = set(ci)
                new_clause.remove(li)
                new_clause.update([x for x in cj if x != lj])
                resolvents.append(frozenset(new_clause))
    return resolvents

def resolution(kb, query):
    clauses = set(kb)
    clauses.add(frozenset([negate_literal(query)]))

    print("Initial Clauses:")
    for c in clauses:
        print(" ", c)
    print("\nResolution Steps:\n")

    new = set()
    iteration = 1

    while True:
        pairs = [(ci, cj) for ci in clauses for cj in clauses if ci != cj]

        for (ci, cj) in pairs:
            resolvents = resolve(ci, cj)
            for r in resolvents:
                if len(r) == 0:
                    print(f"Iteration {iteration}: Derived empty clause → CONTRADICTION")
                    return True
                if r not in clauses:
                    print(f"Iteration {iteration}: Resolving\n  {ci}\n  {cj}\n  => {r}\n")
                    new.add(r)
                    if iteration > 6:     # keep output small
                        return False
                    iteration += 1

        if new.issubset(clauses):
            return False
        clauses |= new

# --------------------- KB in CNF --------------------------

KB = {
    frozenset(["¬Food(x)", "Likes(John,x)"]),
    frozenset(["Food(Apple)"]),
    frozenset(["Food(Vegetables)"]),
    frozenset(["¬Eats(a,b)", "¬Alive(a)", "Food(b)"]),
    frozenset(["Eats(Anil,Peanuts)"]),
    frozenset(["Alive(Anil)"]),
    frozenset(["¬Eats(Anil,x)", "Eats(Harry,x)"]),
    frozenset(["¬Alive(a)", "¬Killed(a)"]),
    frozenset(["Killed(a)", "Alive(a)"])
}

query = "Likes(John,Peanuts)"

result = resolution(KB, query)

if result:
    print("\nConclusion: Likes(John, Peanuts) is TRUE by resolution.")
else:
    print("\nCould not derive contradiction. Query not proven.")


Initial Clauses:
  frozenset({'¬Eats(a,b)', '¬Alive(a)', 'Food(b)'})
  frozenset({'Eats(Harry,x)', '¬Eats(Anil,x)'})
  frozenset({'Likes(John,x)', '¬Food(x)'})
  frozenset({'Eats(Anil,Peanuts)'})
  frozenset({'Alive(a)', 'Killed(a)'})
  frozenset({'Food(Vegetables)'})
  frozenset({'Food(Apple)'})
  frozenset({'Alive(Anil)'})
  frozenset({'¬Likes(John,Peanuts)'})
  frozenset({'¬Alive(a)', '¬Killed(a)'})

Resolution Steps:

Iteration 1: Resolving
  frozenset({'¬Eats(a,b)', '¬Alive(a)', 'Food(b)'})
  frozenset({'Alive(a)', 'Killed(a)'})
  => frozenset({'¬Eats(a,b)', 'Food(b)', 'Killed(a)'})

Iteration 2: Resolving
  frozenset({'Alive(a)', 'Killed(a)'})
  frozenset({'¬Eats(a,b)', '¬Alive(a)', 'Food(b)'})
  => frozenset({'¬Eats(a,b)', 'Food(b)', 'Killed(a)'})

Iteration 3: Resolving
  frozenset({'Alive(a)', 'Killed(a)'})
  frozenset({'¬Alive(a)', '¬Killed(a)'})
  => frozenset({'Killed(a)', '¬Killed(a)'})

Iteration 4: Resolving
  frozenset({'Alive(a)', 'Killed(a)'})
  frozenset({'¬Alive(a)'

In [4]:
# Resolution prover customized for the "John likes peanuts" proof.
# Outputs steps exactly like manual classroom resolution.

def substitute(term, subs):
    if term in subs:
        return subs[term]
    return term

def apply_subs(clause, subs):
    new_clause = []
    for lit in clause:
        pred, args = lit
        new_args = tuple(substitute(a, subs) for a in args)
        new_clause.append((pred, new_args))
    return new_clause

def negate(pred):
    return pred[1:] if pred.startswith("¬") else "¬" + pred

def resolve(c1, c2):
    """Try to resolve c1 and c2. Return (resolvent, substitutions, pivot) or None."""
    for lit1 in c1:
        pred1, args1 = lit1
        for lit2 in c2:
            pred2, args2 = lit2
            if pred1 == negate(pred2):
                subs = {}
                ok = True
                for a, b in zip(args1, args2):
                    if a.startswith("?"):
                        subs[a] = b
                    elif b.startswith("?"):
                        subs[b] = a
                    elif a != b:
                        ok = False
                        break
                if not ok:
                    continue
                c1_new = apply_subs([x for x in c1 if x != lit1], subs)
                c2_new = apply_subs([x for x in c2 if x != lit2], subs)
                resolvent = c1_new + c2_new
                return resolvent, subs, (lit1, lit2)
    return None

# ----------------------- CLAUSES (in literal form) ----------------------------
KB = [
    # 1. ¬food(x) ∨ likes(John,x)
    [("¬food", ("?x",)), ("likes", ("John","?x"))],

    # 2. food(Apple)
    [("food", ("Apple",))],

    # 3. food(Vegetables)
    [("food", ("Vegetables",))],

    # 4. ¬eats(y,z) ∨ killed(y) ∨ food(z)
    [("¬eats", ("?y","?z")), ("killed", ("?y",)), ("food", ("?z",))],

    # 5. eats(Anil,Peanuts)
    [("eats", ("Anil","Peanuts"))],

    # 6. alive(Anil)
    [("alive", ("Anil",))],

    # 7. ¬eats(Anil,w) ∨ eats(Harry,w)
    [("¬eats", ("Anil","?w")), ("eats", ("Harry","?w"))],

    # 8. killed(g) ∨ alive(g)
    [("killed", ("?g",)), ("alive", ("?g",))],

    # 9. ¬alive(k) ∨ ¬killed(k)
    [("¬alive", ("?k",)), ("¬killed", ("?k",))]
]

# Negated Query: ¬likes(John,Peanuts)
query = [("¬likes", ("John","Peanuts"))]
clauses = KB + [query]

# ------------------------- RESOLUTION LOOP ---------------------------
print("Resolution Proof:\n")

used_pairs = set()
step = 1

while True:
    found = False

    for i in range(len(clauses)):
        for j in range(i+1, len(clauses)):
            if (i, j) in used_pairs:
                continue
            used_pairs.add((i, j))

            res = resolve(clauses[i], clauses[j])
            if res:
                resolvent, subs, pivot = res
                print(f"Step {step}: Resolve")
                print("   ", clauses[i])
                print("   ", clauses[j])
                print("   Substitution:", subs)
                print("   Resolvent:", resolvent)
                print()

                if resolvent == []:
                    print("Empty clause {} derived → CONTRADICTION.\n")
                    print("Therefore: likes(John, Peanuts) is TRUE.")
                    raise SystemExit()

                clauses.append(resolvent)
                step += 1
                found = True
                break
        if found:
            break

    if not found:
        print("\nNo more resolutions — proof incomplete.")
        break


Resolution Proof:

Step 1: Resolve
    [('¬food', ('?x',)), ('likes', ('John', '?x'))]
    [('food', ('Apple',))]
   Substitution: {'?x': 'Apple'}
   Resolvent: [('likes', ('John', 'Apple'))]

Step 2: Resolve
    [('¬food', ('?x',)), ('likes', ('John', '?x'))]
    [('food', ('Vegetables',))]
   Substitution: {'?x': 'Vegetables'}
   Resolvent: [('likes', ('John', 'Vegetables'))]

Step 3: Resolve
    [('¬food', ('?x',)), ('likes', ('John', '?x'))]
    [('¬eats', ('?y', '?z')), ('killed', ('?y',)), ('food', ('?z',))]
   Substitution: {'?x': '?z'}
   Resolvent: [('likes', ('John', '?z')), ('¬eats', ('?y', '?z')), ('killed', ('?y',))]

Step 4: Resolve
    [('¬food', ('?x',)), ('likes', ('John', '?x'))]
    [('¬likes', ('John', 'Peanuts'))]
   Substitution: {'?x': 'Peanuts'}
   Resolvent: [('¬food', ('Peanuts',))]

Step 5: Resolve
    [('¬eats', ('?y', '?z')), ('killed', ('?y',)), ('food', ('?z',))]
    [('eats', ('Anil', 'Peanuts'))]
   Substitution: {'?y': 'Anil', '?z': 'Peanuts'}
   Resol

SystemExit: 

C:\ProgramData\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [5]:
from itertools import combinations

# -------------------- Unification ----------------------------

def is_var(x):
    return isinstance(x, str) and x[0].islower()

def unify(x, y, subst):
    if subst is False:
        return False
    if x == y:
        return subst
    if is_var(x):
        return unify_var(x, y, subst)
    if is_var(y):
        return unify_var(y, x, subst)
    if isinstance(x, tuple) and isinstance(y, tuple):
        if len(x) != len(y):
            return False
        for a, b in zip(x, y):
            subst = unify(a, b, subst)
        return subst
    return False

def unify_var(var, x, subst):
    if var in subst:
        return unify(subst[var], x, subst)
    if occurs(var, x, subst):
        return False
    subst[var] = x
    return subst

def occurs(var, x, subst):
    if var == x:
        return True
    if isinstance(x, tuple):
        return any(occurs(var, t, subst) for t in x)
    if x in subst:
        return occurs(var, subst[x], subst)
    return False

# -------------------- Substitution ---------------------------

def apply_subst(term, subst):
    pred, args, sign = term
    new_args = []
    for a in args:
        if isinstance(a, str):
            new_args.append(subst.get(a, a))
        else:
            new_args.append(apply_subst(a, subst))
    return (pred, tuple(new_args), sign)

def apply_subst_clause(clause, subst):
    return [apply_subst(lit, subst) for lit in clause]

# -------------------- Resolution -----------------------------

def resolve(c1, c2):
    resolvents = []
    for i, lit1 in enumerate(c1):
        for j, lit2 in enumerate(c2):

            # Complementary literals?
            if lit1[0] == lit2[0] and lit1[2] != lit2[2]:

                subst = unify(lit1[1], lit2[1], {})
                if subst is False:
                    continue

                new_clause = (
                    c1[:i] + c1[i+1:] +
                    c2[:j] + c2[j+1:]
                )
                new_clause = apply_subst_clause(new_clause, subst)
                new_clause = list(set(new_clause))  # remove duplicates

                resolvents.append(new_clause)
    return resolvents

# -------------------------------------------------------------
#    MINIMAL KNOWLEDGE BASE (3 clauses)
# -------------------------------------------------------------

KB = [

    # John likes any food
    [("Food", ("x",), False), ("Likes", ("John", "x"), True)],

    # Anil eats peanuts
    [("Eats", ("Anil", "Peanuts"), True)],

    # Anything Anil eats is food
    [("Eats", ("Anil", "y"), False), ("Food", ("y",), True)],
]

# Negated goal
goal = [("Likes", ("John", "Peanuts"), False)]

clauses = KB + [goal]

# -------------------------------------------------------------
#   Resolution Loop
# -------------------------------------------------------------

def resolution(clauses):
    new = []

    while True:
        pairs = list(combinations(clauses, 2))

        for (c1, c2) in pairs:
            resolvents = resolve(c1, c2)

            for r in resolvents:
                print("Resolvent:", r)

                if r == []:
                    print("\n✓ Empty clause derived!")
                    print("✓ Therefore: John likes peanuts.")
                    return True

                new.append(r)

        if all(r in clauses for r in new):
            print("\nNo new clauses. Cannot prove.")
            return False

        for r in new:
            if r not in clauses:
                clauses.append(r)

print("RUNNING RESOLUTION...\n")
resolution(clauses)

RUNNING RESOLUTION...

Resolvent: [('Likes', ('John', 'y'), True), ('Eats', ('Anil', 'y'), False)]
Resolvent: [('Food', ('Peanuts',), False)]
Resolvent: [('Food', ('Peanuts',), True)]
Resolvent: [('Likes', ('John', 'y'), True), ('Eats', ('Anil', 'y'), False)]
Resolvent: [('Food', ('Peanuts',), False)]
Resolvent: [('Likes', ('John', 'Peanuts'), True)]
Resolvent: [('Food', ('Peanuts',), True)]
Resolvent: [('Likes', ('John', 'Peanuts'), True)]
Resolvent: [('Eats', ('Anil', 'Peanuts'), False)]
Resolvent: [('Eats', ('Anil', 'Peanuts'), False)]
Resolvent: []

✓ Empty clause derived!
✓ Therefore: John likes peanuts.


True